In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from autogluon.tabular import TabularDataset, TabularPredictor
pd.DataFrame.iteritems = pd.DataFrame.items

In [2]:
train = pd.read_csv('/Volumes/TAEJUN/[0] Study/[9] 데이터분석/playground-series-s4e1/train.csv')
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:
test = pd.read_csv('/Volumes/TAEJUN/[0] Study/[9] 데이터분석/playground-series-s4e1/test.csv')
test.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [4]:
origin = pd.read_csv('/Volumes/TAEJUN/[0] Study/[9] 데이터분석/playground-series-s4e1/Churn_Modelling.csv')
origin.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [5]:
train = pd.concat([train.drop(["id", "Surname"], axis=1), origin.drop(["RowNumber", "Surname"], axis=1)], ignore_index=True)

In [6]:
train.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15674932,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,15749177,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,15694510,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,15741417,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,15766172,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [7]:
test.drop(["id", "Surname"], axis=1, inplace=True)

In [8]:
test.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15773898,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,15782418,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,15807120,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,15808905,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,15607314,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [9]:
train.isna().sum()

CustomerId         0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [10]:
train.isnull().sum()

CustomerId         0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [11]:
train.dropna(inplace=True)

In [12]:
train.isnull().sum()

CustomerId         0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [13]:
train.duplicated().sum()

199

In [14]:
train.drop_duplicates(inplace = True)

In [15]:
train.duplicated().sum()

0

In [16]:
train['HasCrCard'] = train['HasCrCard'].astype('int')
train['IsActiveMember'] = train['IsActiveMember'].astype('int')

test['HasCrCard'] = test['HasCrCard'].astype('int')
test['IsActiveMember'] = test['IsActiveMember'].astype('int')

def add_features(df):
    df['Geo_Gender'] = df['Geography'] + "_" + df['Gender']
    df['AgeGroup'] = df['Age'] // 10 * 10
    df['IsSenior'] = df['Age'].apply(lambda x: 1 if x >= 65 else 0)
    #df['IsYoung'] = df['Age'].apply(lambda x: 1 if x < 25 else 0)
    df['Balance_to_Salary_Ratio'] = (df['Balance'] / df['EstimatedSalary']).astype('float32')
    df['CreditScoreTier'] = pd.cut(df['CreditScore'], bins=[0, 650, 750, 850], labels=['Low', 'Medium', 'High'])
#     df['Products_Per_Tenure'] = (df['NumOfProducts'] / df['Tenure']).astype('float32')
    df['IsActive_by_CreditCard'] = (df['HasCrCard'] * df['IsActiveMember']).astype('float32')
    df['Customer_Status'] = df['Tenure'].apply(lambda x: 'New' if x < 2 else 'Long-term')
    return df

train = add_features(train)
test = add_features(test)

In [17]:
train.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geo_Gender,AgeGroup,IsSenior,Balance_to_Salary_Ratio,CreditScoreTier,IsActive_by_CreditCard,Customer_Status
0,15674932,668,France,Male,33.0,3,0.00,2,1,0,181449.97,0,France_Male,30.0,0,0.000000,Medium,0.0,Long-term
1,15749177,627,France,Male,33.0,1,0.00,2,1,1,49503.50,0,France_Male,30.0,0,0.000000,Low,1.0,New
2,15694510,678,France,Male,40.0,10,0.00,2,1,0,184866.69,0,France_Male,40.0,0,0.000000,Medium,0.0,Long-term
3,15741417,581,France,Male,34.0,2,148882.54,1,1,1,84560.88,0,France_Male,30.0,0,1.760655,Low,1.0,Long-term
4,15766172,716,Spain,Male,33.0,5,0.00,2,1,1,15068.83,0,Spain_Male,30.0,0,0.000000,Medium,1.0,Long-term


In [18]:
test.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geo_Gender,AgeGroup,IsSenior,Balance_to_Salary_Ratio,CreditScoreTier,IsActive_by_CreditCard,Customer_Status
0,15773898,586,France,Female,23.0,2,0.00,2,0,1,160976.75,France_Female,20.0,0,0.000000,Low,0.0,Long-term
1,15782418,683,France,Female,46.0,2,0.00,1,1,0,72549.27,France_Female,40.0,0,0.000000,Medium,0.0,Long-term
2,15807120,656,France,Female,34.0,7,0.00,2,1,0,138882.09,France_Female,30.0,0,0.000000,Medium,0.0,Long-term
3,15808905,681,France,Male,36.0,8,0.00,1,1,0,113931.57,France_Male,30.0,0,0.000000,Medium,0.0,Long-term
4,15607314,752,Germany,Male,38.0,10,121263.62,1,1,0,139431.00,Germany_Male,30.0,0,0.869703,High,0.0,Long-term


In [19]:
train = TabularDataset(train)
test = TabularDataset(test)

In [20]:
train['IsActive_by_CreditCard'].isin([np.inf]).sum()

0

In [ ]:
automl = TabularPredictor(label='Exited', problem_type='binary', eval_metric='roc_auc')
automl.fit(train, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240112_154035"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240112_154035/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20240112_154035/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

	0.8756	 = Validation score   (roc_auc)
	42.18s	 = Training   runtime
	5.44s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 389.16s of the 688.7s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=2.94%)
		ray::_ray_fit() (pid=9522, ip=127.0.0.1)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 402, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 854, in fit
    out = self._fit(**kwargs)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/tabular/models/catboost/catboost_model.py", line 118, in _fit
    X = Pool(data=X, label=y, cat_features=cat_feat

2024-01-13 00:44:26,920	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-01-13 00:44:26,934	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::_ray_fit() (pid=9524, ip=127.0.0.1)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 402, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 854, in fit
    out = self._fit(**kwargs)
  File "/Users/tj/anaconda3/envs/ML/lib/python3.9/site-packages/autogluon/tabular/models/catboost/catboost_model.py", line 118, in _fit
    X = Pool(data=X, label=y, cat_features=cat_

In [ ]:
automl.leaderboard()

In [ ]:
prediction = automl.predict_proba(test)

data_submit = pd.read_csv('/Volumes/TAEJUN/[0] Study/[9] 데이터분석/playground-series-s4e1/sample_submission.csv')
data_submit.Exited = prediction[1]
data_submit[['id', 'Exited']].to_csv('PG_s4e1_autogluon.csv', index=False)